# **CH01.3. fastText**

#### **`Paper Info`** : Enriching Word Vectors with Subword Information
##### $ \hspace{1.75cm} - $ Piotr Bojanowski, Edouard Grave, Armand Joulin, Tomas Mikolov et al. (TACL/2017)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|-------------|
| Research topic | 단어를 character n-gram의 합으로 표현하는 subword-aware Skip-gram으로 희귀 단어와 OOV의 의미를 포착 |
| Key findings | $ \cdot{} $ character n-gram 합성으로 형태 변화가 많은 언어에서 Skip-gram 대비 유의한 향상 <br> $ \cdot{} $ OOV 단어도 subword의 합으로 즉시 임베딩 가능 <br> $ \cdot{} $ 속도는 Skip-gram 대비 $ \sim{}1.5\times{} $ 느리지만 메모리와 학습 효율은 해시 버킷으로 제어 가능 |
| Contributions | $ \cdot{} $ word vector를 subword 구성요소의 합으로 재정의한 scoring 함수 제안 <br> $ \cdot{} $ 해시 버킷 기반 n-gram 파라미터화로 대규모 어휘에 확장 <br> $ \cdot{} $ 다국어 Wikipedia 기반 및 뉴스 corpus에서 similarity, analogy, LM에서 일관된 이득 보고 |

<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ Word2Vec 모델은 word 타입별 파라미터로 형태학적 변이와 어근 공유를 직접 활용하지 못함
##### $ \hspace{0.15cm} \cdot{} $ 희귀 단어와 OOV는 학습 빈도 부족 및 미등록으로 임베딩 품질 저하
##### $ \hspace{0.15cm} \cdot{} $ 형태가 풍부한 언어에서 표면형 분산이 커져 분산표현의 데이터 효율이 낮음

#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ 단어 내부의 subword 단서를 이용해 희귀 단어와 OOV의 의미를 추론
##### $ \hspace{0.15cm} \cdot{} $ 기존 Skip-gram의 학습 목적과 최적화 절차를 유지하면서 표현력만 확장
##### $ \hspace{0.15cm} \cdot{} $ 해시 기반 설계로 메모리와 시간 복잡도를 실용 범위로 제한

#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ 단어를 boundary가 포함된 character n-gram 집합의 합으로 스코어링하는 Subword Skip-gram 제안
##### $ \hspace{0.15cm} \cdot{} $ 각 n-gram을 해시 버킷에 매핑해 고정 크기 파라미터 테이블로 효율을 확보
##### $ \hspace{0.15cm} \cdot{} $ Negative sampling으로 확률 근사해 대규모 말뭉치에 적합

<b></b>

> ## **방법론(Method)**

#### **(1) SGNS(Subword Skip-gram with Negative Sampling)** :
##### $ \hspace{0.15cm} \cdot{} $ 중심 단어 $ \, w_{t} $와 주변 단어 집합 $ \, \mathcal{C}_{t} $이 주어졌을 때, 각 문맥 단어 $ \, c_{t,j} $가 양성쌍(positive pair)일 확률 $ \, p_{\theta{}}(y=1|w_{t}, c_{t,j}) $을 최대화하는 것을 목표
##### $ \hspace{0.3cm} \Rightarrow{} \displaystyle{} \theta{}^{*} = \argmax_{\theta{}} \prod_{t=1}^{T} \prod_{\substack{-c \le{} j \le{} c \\ j \neq{} 0}} p_{\theta{}}(y=1|w_{t}, c_{t,j}) \;\; \text{ where } \, y \in{} \{0,1\}, \;\; y=1 : \text{positive pair} $
##### $ \hspace{0.45cm} \text{and } \, T : \text{token size}, \;\; c : \text{window radius} $
##### $ \hspace{0.15cm} \cdot{} $ 양성쌍 확률 $ \, p_{\theta{}}(y=1|w_{t}, c_{t,j}) $은 binary logistic probability로 정의하며, 단어 내 character n-gram 임베딩 합을 입력으로 사용
##### $ \hspace{0.3cm} \Rightarrow{} p_{\theta{}}(y=1 | w_{t}, c_{t,j}) = \sigma{}\big({}\mathbf{u}_{c_{t,j}}^{\top} \sum_{g \in{} \mathcal{G}(w_{t})} \mathbf{z}_{g} \big) $
##### $ \hspace{0.45cm} \text{where } \, \mathcal{G}(w_{t}) : \text{set of character n-grams in } w_{t}, \;\; \mathbf{z}_{g} \in{} \mathbb{R}^{d} : \text{embedding of n-gram } g $
##### $ \hspace{0.45cm} \text{and } \, \mathbf{u}_{c_{t,j}} \in{} \mathbb{R}^{d} : \text{output(context) embedding}, \;\; \sigma{}(x) = \frac{1}{1+\exp{}(-x)} $ 
##### $ \hspace{0.15cm} \cdot{} $ 모든 토큰 시점 $ t $과 윈도우 내 문맥 $ j $에 대해, 양성·음성 로지스틱 손실의 합을 최대화하도록 SGNS의 목적함수를 정의
##### $ \hspace{0.3cm} \Rightarrow{} J^{*} = \displaystyle{} \argmax_{\theta{}} \sum_{t=1}^{T} \sum_{\substack{-c \le{} j \le{} c \\ j \neq{} 0}} \Bigg[ \log{}\sigma{}\big({}\mathbf{u}_{c_{t,j}}^{\top} \sum_{g \in{} \mathcal{G}(w_{t})} \mathbf{z}_{g} \big) + \sum_{k=1}^{K} \log{}\sigma{}\big({}-\mathbf{u}_{\tilde{c}_{t,j,k}}^{\top} \sum_{g \in{} \mathcal{G}(w_{t})} \mathbf{z}_{g} \big) \Bigg] $
##### $ \hspace{0.45cm} \text{where } \, \tilde{c}_{t,j,k} : \text{k-th negative sample}, \;\; K : \text{\#negatives per positive} $

<b></b>

> ## **실험(Experiments)**

##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | Wikipedia dumps (9 languages) |
| Languages | Ar, Cz, De, En, Es, Fr, It, Ro, Ru |
| Vocabulary size($ V $)| ? |
| Token size($ T $) | ? |
| Evaluation dataset | Word similarity / Analogy / LM |
##### $ \hspace{0.15cm} \cdot{} $ 모델
| Item | Description |
|------|--------------|
| Model type | Subword Skip-gram (fastText) |
| Architecture | $ \cdot{} $ Input: character n-gram sum (3–6) with boundary <br> $ \cdot{} $ Output: word-level context vector <br> $ \cdot{} $ Hash bucket: $ 2 \times 10^{6} $ |
| Hyper parameter | $ \cdot{} $ Embedding dim($ d $): 300 <br> $ \cdot{} $ Window size: 1–5 (uniform) <br> $ \cdot{} $ Negatives: 5 (√unigram) <br> $ \cdot{} $ Subsampling: $ 10^{-4} $ <br> $ \cdot{} $ Optimizer: SGD (linear decay) <br> $ \cdot{} $ Epochs: 5 |
| Parameter size | ? |
##### $ \hspace{0.15cm} \cdot{} $ 훈련 및 평가
| Item | Description |
|------|--------------|
| Loss function | Negative log-likelihood (binary logistic) |
| Evaluation metric | Spearman corr. / Analogy accuracy / LM perplexity |
| Baseline models | Skip-gram, CBOW |
| Training hardware | Multi-threaded CPU |
| Training time | $ \sim{}1.5\times{} $ slower than Skip-gram |

#### **(2) 실험 결과** :
##### $ \hspace{0.15cm} \cdot{} $ 형태가 풍부한 언어에서 word-only Skip-gram 대비 word+subword가 similarity와 analogy에서 일관된 향상
##### $ \hspace{0.15cm} \cdot{} $ OOV 단어에 대해 학습된 n-gram의 선형 합으로 즉시 임베딩을 산출해 다운스트림 품질을 유지
##### $ \hspace{0.15cm} \cdot{} $ LM 평가에서 subword 모델은 희귀 표면형을 좀 더 일반화함

<b></b>

> ## **결론(Conclusion)**

#### **(1)** SISG은 단어 임베딩을 character n-gram의 합으로 정의해 형태 정보와 어근 공유를 직접 활용함

#### **(2)** Subword Skip-gram은 희귀 단어와 OOV에서 강인한 일반화를 제공함

#### **(3)** 해시 버킷과 negative sampling으로 대규모 말뭉치에서도 실용적 복잡도를 유지할 수 있음

#### **(4)** 적절한 n-gram 길이 선택 문제는 남아 있으며 과제 특성과 언어에 따라 검증이 필요함 

<b></b>

> ## **부록(Appendix)**

<b></b>

> ## **비평(Commentary)**

<b></b>

> ## **참고 문헌(Reference)**

#### **(1) 관련 후속 논문** :
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**

#### **(2) 기타 참고 자료** :
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**